In [ ]:
from openpyxl import Workbook, load_workbook
from openpyxl.writer.excel import save_virtual_workbook
import os
import pandas as pd

In [142]:
class InputAdaptater:

    BASE_ELEMENTS_ROW = 17 # Location of base's elements
    UNIT_ROW = 18 # Location of units
    CURVE = 19 # Location of Curve's interpolations
    METADATA_COL = "A"
    POINT_N_COL = "B"
    DATE_COL = "D"

    def __init__(self, input="/input/model.xlsx") -> None:
        self.wb = load_workbook(os.getcwd()+ input, data_only=True)
        self.json = {}

    def _getPointsWithDates(self, sheet_name):
        """
            Return dict (row_number, point_no, date) of all points and associate's date
        """
        ws = self.wb[sheet_name]
        return [{
            "row": point.row, 
            "point_n": point.value, 
            "date": ws[self.DATE_COL+str(point.row)].value
            } for point in ws[self.POINT_N_COL] if point.value is not None and type(point.value) is int]

    def _getSpecifications(self, sheet_name):
        """
        Return dict (column_id, specification, unit, interpolation) at BASE_ELEMENTS_ROW for a given sheetname
        Ignore the firsts 2 elements because they always not belong to specifications
        """
        ws = self.wb[sheet_name]
        return [{
            "column": be.column, 
            "specification_name": be.value, 
            "unit": ws.cell(row=self.UNIT_ROW, column=be.column).value, 
            "interpolation": ws.cell(row=self.CURVE, column=be.column).value
        } for be in ws[self.BASE_ELEMENTS_ROW] if be.value is not None][2:]

    def _getValuesBySpecificiation(self, specification, dates, sheet_name):
        ws = self.wb[sheet_name]

        return [(date["row"], ws.cell(row=date["row"], column=specification["column"]).value) for date in dates if ws.cell(row=date["row"], column=specification["column"]).value is not None]

    def _getSpectificationsWithInterpolations(self, sheet_name):
        """
            return a dict with specifications and interpolations
        """
        result = []
        dates = self._getPointsWithDates(sheet_name)
        
        for specification in self._getSpecifications(sheet_name):
            return specification["specification_name"], self._getValuesBySpecificiation(specification, dates, sheet_name)
            #for date in dates:
            #    if specifiction["unit"] == "CONST":
            #        date["value"] = 

    def _getBaseElementMetaData(self, sheet_name):
        """
        Return array of tuples (row_id, metadata_name, metadata_value) for a given sheetname
        """
        ws = self.wb.get_sheet_by_name(sheet_name)
        return [(cell.row, cell.value, ws['B'][cell.row].value) for cell in ws[self.METADATA_COL] if (cell.value is not None and cell.row < self.BASE_ELEMENTS_ROW)]

    def _getBaseElementsSheetNames(self):
        pass

    def _getRawDataStorage(self):
        """
            Return JSON storage of the input
        """
        return self._getPointsWithDates(self.wb.sheetnames[1])


In [143]:
ia = InputAdaptater()
ia._getRawDataStorage()

[{'row': 22, 'point_n': 0, 'date': datetime.datetime(2022, 7, 1, 0, 0)},
 {'row': 23, 'point_n': 1, 'date': datetime.datetime(2023, 7, 1, 0, 0)},
 {'row': 24, 'point_n': 2, 'date': datetime.datetime(2024, 7, 1, 0, 0)},
 {'row': 25, 'point_n': 3, 'date': datetime.datetime(2025, 7, 1, 0, 0)},
 {'row': 26, 'point_n': 4, 'date': datetime.datetime(2026, 7, 1, 0, 0)},
 {'row': 27, 'point_n': 5, 'date': datetime.datetime(2027, 7, 1, 0, 0)},
 {'row': 28, 'point_n': 6, 'date': datetime.datetime(2028, 7, 1, 0, 0)},
 {'row': 29, 'point_n': 7, 'date': datetime.datetime(2029, 7, 1, 0, 0)},
 {'row': 30, 'point_n': 8, 'date': datetime.datetime(2030, 7, 1, 0, 0)},
 {'row': 31, 'point_n': 9, 'date': datetime.datetime(2031, 7, 1, 0, 0)},
 {'row': 32, 'point_n': 10, 'date': datetime.datetime(2032, 7, 1, 0, 0)},
 {'row': 33, 'point_n': 11, 'date': datetime.datetime(2033, 7, 1, 0, 0)},
 {'row': 34, 'point_n': 12, 'date': datetime.datetime(2034, 7, 1, 0, 0)},
 {'row': 35, 'point_n': 13, 'date': datetime.dat

In [ ]:
import math
import numpy as np
# Créer un array de count(_getDates) elements à 0
np.linspace(0,10,20)
# map cette array avec sa fonction d'interpolation